# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [74]:
import pandas as pd
import numpy as np
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [ ]:
# 2) seu código aqui
sinasc.isna().sum()

In [144]:
# 3) seu código aqui
df_sinasc = pd.read_csv('SINASC_RO_2019.csv')[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]


In [145]:
# 4) seu código aqui
df_sinasc.dropna(subset='APGAR5',inplace=True)
df_sinasc.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [146]:
# 5) seu código aqui
df_sinasc['ESTCIVMAE'].fillna(9, inplace=True)
df_sinasc.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [147]:
# 6) Seu código aqui
df_sinasc['QTDFILVIVO'].fillna(0, inplace=True)

In [148]:
# 7) seu código aqui
# por se tratar de um assunto tão sensível, acho importante que os dados sejam reais ao máximo
# dropando dados faltantes ao invéz de preencher

df_sinasc.dropna(inplace=True)
df_sinasc.isna().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [149]:
# 8) seu código aqui

# O Apgar possui uma classificação indicando se o bebê passou por asfixia:

# Entre 8 e 10 está em uma faixa 'normal'.
# Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
# Entre 4 e 5 significa 'asfixia moderada'.
# Entre 0 e 3 significa 'asfixia severa'.

df_sinasc.loc[df_sinasc['APGAR5'] >= 8, 'APGAR5_CAT'] = 'normal'
df_sinasc.loc[(df_sinasc['APGAR5'] > 5)&(df_sinasc['APGAR5'] < 8), 'APGAR5_CAT'] = 'asfixia leve'
df_sinasc.loc[(df_sinasc['APGAR5'] > 3)&(df_sinasc['APGAR5'] < 6), 'APGAR5_CAT'] = 'asfixia moderada'
df_sinasc.loc[df_sinasc['APGAR5'] < 4, 'APGAR5_CAT'] = 'asfixia severa'


In [157]:
df_sinasc.APGAR5_CAT.value_counts()

APGAR5_CAT
normal              24946
asfixia leve          302
asfixia severa         68
asfixia moderada       59
Name: count, dtype: int64

In [164]:
# 9) seu código aqui
df_sinasc.columns = ['local_nascimento', 'idade_mae', 'estado_civil_mae', 'escolaridade_mae', 'quantidade_filho_vivo', 'gestaçao', 'gravidez', 'consultas', 'apagar_5', 'apagar_5_categoria']
df_sinasc

,local_nascimento,idade_mae,estado_civil_mae,escolaridade_mae,quantidade_filho_vivo,gestaçao,gravidez,consultas,apagar_5,apagar_5_categoria
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
